In [6]:
# %matplotlib inline
from new_methods.new_methods import My_NuScenes
import numpy as np
import pythreejs as pjs

nusc = My_NuScenes(version='v1.0-mini', dataroot='/data/datasets/nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.223 seconds.
Reverse indexing ...
Done reverse indexing in 0.0 seconds.


In [10]:
idx = 79
radar_pc = nusc.make_points_list('r', idx)
lidar_pc = nusc.make_points_list('l', idx)
total_pc = np.vstack((radar_pc, lidar_pc)).astype(np.float32)
pc_num = (len(radar_pc),len(lidar_pc))
pc_num

(159, 34400)

In [11]:
def visualize_3d(xyz, pc_num, rgb_radar=None, rgb_lidar=None,
                 size_radar=0.1, size_lidar=0.2, height=800, width=800):
    # pts
    radar_pc = xyz[:pc_num[0]]
    lidar_pc = xyz[pc_num[0]:]

    # radar pc
    radar_points_buf = pjs.BufferAttribute(array=radar_pc)
    radar_colors_buf = pjs.BufferAttribute(array=np.array([rgb_radar] * pc_num[0], dtype=np.float32))
    radar_geometry = pjs.BufferGeometry(attributes={'position': radar_points_buf, 'color': radar_colors_buf})
    radar_material = pjs.PointsMaterial(vertexColors='VertexColors', size=size_radar)
    radar_pointCloud = pjs.Points(geometry=radar_geometry, material=radar_material)

    # lidar pc
    lidar_points_buf = pjs.BufferAttribute(array=lidar_pc)
    lidar_colors_buf = pjs.BufferAttribute(array=np.array([rgb_lidar] * pc_num[1], dtype=np.float32))
    lidar_geometry = pjs.BufferGeometry(attributes={'position': lidar_points_buf, 'color': lidar_colors_buf})
    lidar_material = pjs.PointsMaterial(vertexColors='VertexColors', size=size_lidar)
    lidar_pointCloud = pjs.Points(geometry=lidar_geometry, material=lidar_material)

    pythreejs_camera = pjs.PerspectiveCamera(up=[1, 0, 1], children=[pjs.DirectionalLight(color='black', intensity=0.5)])
    pythreejs_camera.rotateX(np.pi/4)
    pythreejs_camera.position = (-15., 0., 30.)
    
    scene = pjs.Scene(children=[radar_pointCloud, lidar_pointCloud, pythreejs_camera, pjs.AmbientLight(color='#777777')])
    axes = pjs.AxesHelper(size=3)
    scene.add(axes)
        
    control = pjs.OrbitControls(controlling=pythreejs_camera)
    renderer = pjs.Renderer(camera=pythreejs_camera, scene=scene, width=width, height=height, preserveDrawingBuffer=True, controls=[control])
    
    return renderer


In [13]:
renderer = visualize_3d(total_pc, pc_num, rgb_radar=[1,0,0], rgb_lidar=[0,0,1], size_radar=5,
                        size_lidar=0.1)
renderer

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='black', intensity=0.5, position=(0.0, 0.0,…